In [1]:
import pandas as pd

# Load Billboard Hot 100 songs
df_songs_hot_100 = pd.read_csv('/Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_hot100_songs_unique.csv')

# Load Billboard 200 albums
df_albums_hot_100 = pd.read_csv('/Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_200_albums_unique.csv')

# Display basic info about the dataframes
print(f"Songs dataset shape: {df_songs_hot_100.shape}")
print(f"Albums dataset shape: {df_albums_hot_100.shape}")


Songs dataset shape: (32363, 5)
Albums dataset shape: (38085, 5)


In [5]:
import pandas as pd
import requests
import time
import re
from urllib.parse import quote
import os

cache_path = os.path.expanduser('~/artist_verification_cache.csv')

def check_musicbrainz(artist_name, max_retries=2):
    """Check if artist exists in MusicBrainz"""
    base_url = "https://musicbrainz.org/ws/2/artist/"
    encoded_name = quote(artist_name)
    url = f"{base_url}?query=artist:{encoded_name}&fmt=json"
    
    for attempt in range(max_retries):
        try:
            headers = {'User-Agent': 'BillboardDataCleaner/1.0 (educational-project)'}
            response = requests.get(url, headers=headers, timeout=10)
            
            if response.status_code == 200:
                data = response.json()
                if 'artists' in data and len(data['artists']) > 0:
                    for artist in data['artists']:
                        if artist['name'].lower() == artist_name.lower():
                            return True, 100
                        if 'aliases' in artist:
                            for alias in artist['aliases']:
                                if alias.get('name', '').lower() == artist_name.lower():
                                    return True, 100
                    return False, 0
                else:
                    return False, 0
            elif response.status_code == 503:
                time.sleep(3)
                continue
            else:
                return False, 0
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2)
                continue
            return False, 0
    
    return False, 0

def check_musicbrainz_variants(artist_name, max_retries=2):
    """Check with & and 'and' variants"""
    exists, confidence = check_musicbrainz(artist_name, max_retries)
    if exists:
        return exists, confidence
    
    if '&' in artist_name:
        variant = artist_name.replace('&', 'and')
        exists, confidence = check_musicbrainz(variant, max_retries)
        if exists:
            return exists, confidence
    elif ' and ' in artist_name.lower():
        variant = re.sub(r'\s+[Aa]nd\s+', ' & ', artist_name)
        exists, confidence = check_musicbrainz(variant, max_retries)
        if exists:
            return exists, confidence
    
    return False, 0

def smart_split_with_api_check(df, use_cache=True, cache_file=None):
    """Split performers using API verification"""
    if use_cache and cache_file:
        try:
            cache_df = pd.read_csv(cache_file)
            artist_cache = dict(zip(cache_df['artist'], cache_df['is_real_artist']))
            print(f"✓ Loaded {len(artist_cache)} entries from cache\n")
        except:
            artist_cache = {}
    else:
        artist_cache = {}
    
    rows = []
    api_calls = 0
    
    for idx, row in df.iterrows():
        performer = row['performer']
        
        if not re.search(r'&|[Aa]nd|[Ff]eat|[Ff]t\.|[Ww]ith|X', performer):
            rows.append(row.to_dict())
            continue
        
        if performer not in artist_cache:
            # SHOW WHAT WE'RE CHECKING
            print(f"[{idx}/{len(df)}] Checking: {performer[:60]}")
            
            is_real, confidence = check_musicbrainz_variants(performer)
            artist_cache[performer] = is_real
            api_calls += 1
            
            # Show result
            if is_real:
                print(f"  ✓ Single artist/band")
            else:
                print(f"  ✗ Will split")
            
            time.sleep(1.5)
        else:
            is_real = artist_cache[performer]
        
        if is_real:
            rows.append(row.to_dict())
            continue
        
        # Split logic
        performers_list = None
        
        if re.search(r'\s+[Ff]eatur(ing|e)\s+', performer) or re.search(r'\s+[Ff]t\.?\s+', performer):
            performers_list = re.split(r'\s+[Ff]eatur(?:ing|e)\s+|\s+[Ff]t\.?\s+', performer)
        elif re.search(r'\s+[Ww]ith\s+', performer):
            performers_list = re.split(r'\s+[Ww]ith\s+', performer)
        elif ' X ' in performer or ' x ' in performer:
            performers_list = re.split(r'\s+[Xx]\s+', performer)
        elif ',' in performer:
            parts = [p.strip() for p in performer.split(',')]
            if len(parts) > 1 and (' & ' in parts[-1] or ' And ' in parts[-1]):
                last_part_split = re.split(r'\s+&\s+|\s+[Aa]nd\s+', parts[-1])
                performers_list = parts[:-1] + last_part_split
            else:
                performers_list = parts
        elif ' & ' in performer or ' And ' in performer or ' and ' in performer:
            performers_list = re.split(r'\s+&\s+|\s+[Aa]nd\s+', performer)
        
        if performers_list and len(performers_list) > 1:
            verified_artists = []
            
            for p in performers_list:
                p = p.strip()
                if not p:
                    continue
                
                if p not in artist_cache:
                    print(f"    Verifying: {p[:40]}")
                    is_real, confidence = check_musicbrainz_variants(p)
                    artist_cache[p] = is_real
                    api_calls += 1
                    time.sleep(1.5)
                
                verified_artists.append(p)
            
            for artist in verified_artists:
                new_row = row.to_dict()
                new_row['performer'] = artist
                new_row['original_performer'] = performer
                rows.append(new_row)
        else:
            rows.append(row.to_dict())
        
        # Save cache every 20 new API calls
        if use_cache and cache_file and api_calls % 20 == 0:
            try:
                cache_df = pd.DataFrame([
                    {'artist': k, 'is_real_artist': v} 
                    for k, v in artist_cache.items()
                ])
                cache_df.to_csv(cache_file, index=False)
                print(f"\n💾 Cache saved: {len(artist_cache)} entries, {api_calls} new API calls this run\n")
            except:
                pass
    
    # Final save
    if use_cache and cache_file:
        try:
            cache_df = pd.DataFrame([
                {'artist': k, 'is_real_artist': v} 
                for k, v in artist_cache.items()
            ])
            cache_df.to_csv(cache_file, index=False)
            print(f"\n✅ Complete! Final cache: {len(artist_cache)} total entries")
        except:
            pass
    
    return pd.DataFrame(rows)

print("="*60)
print("Running with VERBOSE output so you can see progress...")
print(f"Cache: {cache_path}")
print("="*60 + "\n")

df_songs_collaborations_split = smart_split_with_api_check(
    df_songs_hot_100,
    use_cache=True,
    cache_file=cache_path
)

print(f"\n{'='*60}")
print(f"RESULTS:")
print(f"{'='*60}")
print(f"Original: {len(df_songs_hot_100):,} rows")
print(f"New:      {len(df_songs_collaborations_split):,} rows")
print(f"Added:    {len(df_songs_collaborations_split) - len(df_songs_hot_100):,} rows")
print(f"{'='*60}\n")

# Show examples
if 'original_performer' in df_songs_collaborations_split.columns:
    splits = df_songs_collaborations_split[
        df_songs_collaborations_split['original_performer'].notna()
    ][['title', 'performer', 'original_performer']].head(15)
    
    if len(splits) > 0:
        print("Sample splits:")
        print(splits.to_string(index=False))


Running with VERBOSE output so you can see progress...
Cache: /Users/jamesemcnally/artist_verification_cache.csv

✓ Loaded 280 entries from cache


💾 Cache saved: 280 entries, 0 new API calls this run


💾 Cache saved: 280 entries, 0 new API calls this run


💾 Cache saved: 280 entries, 0 new API calls this run


💾 Cache saved: 280 entries, 0 new API calls this run


💾 Cache saved: 280 entries, 0 new API calls this run


💾 Cache saved: 280 entries, 0 new API calls this run


💾 Cache saved: 280 entries, 0 new API calls this run


💾 Cache saved: 280 entries, 0 new API calls this run


💾 Cache saved: 280 entries, 0 new API calls this run


💾 Cache saved: 280 entries, 0 new API calls this run


💾 Cache saved: 280 entries, 0 new API calls this run


💾 Cache saved: 280 entries, 0 new API calls this run


💾 Cache saved: 280 entries, 0 new API calls this run


💾 Cache saved: 280 entries, 0 new API calls this run


💾 Cache saved: 280 entries, 0 new API calls this run


💾 Cache saved: 280 entries,

In [6]:
# Save to CSV
output_path = '/Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_hot100_songs_collaborations_split.csv'

df_songs_collaborations_split.to_csv(output_path, index=False)

print(f"✅ Saved to: {output_path}")
print(f"📊 Rows: {len(df_songs_collaborations_split):,}")
print(f"📋 Columns: {list(df_songs_collaborations_split.columns)}")


✅ Saved to: /Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_hot100_songs_collaborations_split.csv
📊 Rows: 37,337
📋 Columns: ['title', 'performer', 'peak_pos', 'wks_on_chart', 'first_charting_year', 'original_performer']


In [8]:
# Process albums dataframe (will reuse cache from songs!)
print("="*60)
print("Processing ALBUMS dataframe...")
print(f"Cache: {cache_path} (reusing from songs!)")
print("="*60 + "\n")

df_albums_collaborations_split = smart_split_with_api_check(
    df_albums_hot_100,
    use_cache=True,
    cache_file=cache_path  # SAME cache as songs - this is the magic!
)

print(f"\n{'='*60}")
print(f"ALBUMS RESULTS:")
print(f"{'='*60}")
print(f"Original: {len(df_albums_hot_100):,} rows")
print(f"New:      {len(df_albums_collaborations_split):,} rows")
print(f"Added:    {len(df_albums_collaborations_split) - len(df_albums_hot_100):,} rows")
print(f"{'='*60}\n")

# Show examples
if 'original_performer' in df_albums_collaborations_split.columns:
    splits = df_albums_collaborations_split[
        df_albums_collaborations_split['original_performer'].notna()
    ][['title', 'performer', 'original_performer']].head(15)
    
    if len(splits) > 0:
        print("Sample album splits:")
        print(splits.to_string(index=False))


Processing ALBUMS dataframe...
Cache: /Users/jamesemcnally/artist_verification_cache.csv (reusing from songs!)

✓ Loaded 8751 entries from cache


💾 Cache saved: 8751 entries, 0 new API calls this run


💾 Cache saved: 8751 entries, 0 new API calls this run


💾 Cache saved: 8751 entries, 0 new API calls this run


💾 Cache saved: 8751 entries, 0 new API calls this run


💾 Cache saved: 8751 entries, 0 new API calls this run


💾 Cache saved: 8751 entries, 0 new API calls this run


💾 Cache saved: 8751 entries, 0 new API calls this run


💾 Cache saved: 8751 entries, 0 new API calls this run


💾 Cache saved: 8751 entries, 0 new API calls this run


💾 Cache saved: 8751 entries, 0 new API calls this run


💾 Cache saved: 8751 entries, 0 new API calls this run


💾 Cache saved: 8751 entries, 0 new API calls this run


💾 Cache saved: 8751 entries, 0 new API calls this run


💾 Cache saved: 8751 entries, 0 new API calls this run


💾 Cache saved: 8751 entries, 0 new API calls this run


💾 Cache saved

In [9]:
# Save albums dataframe to CSV
output_path = '/Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_200_albums_collaborations_split.csv'

df_albums_collaborations_split.to_csv(output_path, index=False)

print(f"✅ Saved to: {output_path}")
print(f"📊 Rows: {len(df_albums_collaborations_split):,}")
print(f"📋 Columns: {list(df_albums_collaborations_split.columns)}")

✅ Saved to: /Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_200_albums_collaborations_split.csv
📊 Rows: 38,823
📋 Columns: ['title', 'performer', 'peak_pos', 'wks_on_chart', 'first_charting_year', 'original_performer']


In [10]:
def normalize_performer(name):
    """Clean and standardize performer names"""
    if pd.isna(name):
        return ""
    name = str(name)
    name = name.strip()  # Remove leading/trailing spaces
    name = ' '.join(name.split())  # Normalize internal spaces
    name = name.lower()  # Lowercase for comparison
    return name

# Apply normalization to songs dataframe
df_songs_collaborations_split['performer_normalized'] = df_songs_collaborations_split['performer'].apply(normalize_performer)

print("✅ Songs dataframe normalized")
print(f"Total songs: {len(df_songs_collaborations_split):,}")
print(f"Unique performers: {df_songs_collaborations_split['performer_normalized'].nunique():,}")

# Show examples of normalization
print("\nExamples of normalization:")
sample = df_songs_collaborations_split[['performer', 'performer_normalized']].drop_duplicates().head(10)
print(sample.to_string(index=False))


✅ Songs dataframe normalized
Total songs: 37,337
Unique performers: 9,429

Examples of normalization:
       performer performer_normalized
   Tommy Edwards        tommy edwards
   Conway Twitty        conway twitty
    The Elegants         the elegants
Domenico Modugno     domenico modugno
    Ricky Nelson         ricky nelson
    The Platters         the platters
     Lloyd Price          lloyd price
   The Chipmunks        the chipmunks
   David Seville        david seville
 The Teddy Bears      the teddy bears


In [11]:
# Apply normalization to albums dataframe
df_albums_collaborations_split['performer_normalized'] = df_albums_collaborations_split['performer'].apply(normalize_performer)

print("✅ Albums dataframe normalized")
print(f"Total albums: {len(df_albums_collaborations_split):,}")
print(f"Unique performers: {df_albums_collaborations_split['performer_normalized'].nunique():,}")

# Show examples of normalization
print("\nExamples of normalization:")
sample = df_albums_collaborations_split[['performer', 'performer_normalized']].drop_duplicates().head(10)
print(sample.to_string(index=False))

✅ Albums dataframe normalized
Total albums: 38,823
Unique performers: 10,032

Examples of normalization:
                      performer            performer_normalized
Herb Alpert & The Tijuana Brass herb alpert & the tijuana brass
 Paul Mauriat And His Orchestra  paul mauriat and his orchestra
      Diana Ross & The Supremes       diana ross & the supremes
                     Soundtrack                      soundtrack
                    The Monkees                     the monkees
          The Mamas & The Papas           the mamas & the papas
                    The Beatles                     the beatles
                  Bobbie Gentry                   bobbie gentry
                  Frank Sinatra                   frank sinatra
                   The Supremes                    the supremes


In [14]:
# Check songs dataframe for remaining collaborations
print("="*60)
print("SONGS DATAFRAME - Checking for unsplit collaborations")
print("="*60)

songs_collabs = df_songs_collaborations_split[
    df_songs_collaborations_split['performer'].str.contains(
        r'&|[Aa]nd|[Ff]eat|[Ff]t\.|[Ww]ith|X|,', 
        na=False, 
        regex=True
    )
]

print(f"\nPotential unsplit collaborations in SONGS: {len(songs_collabs):,}")
print(f"Unique performers: {songs_collabs['performer'].nunique():,}")

if len(songs_collabs) > 0:
    print("\nSample potentially unsplit songs performers:")
    print(songs_collabs['performer'].value_counts().head(50))

print("\n" + "="*60)
print("ALBUMS DATAFRAME - Checking for unsplit collaborations")
print("="*60)

albums_collabs = df_albums_collaborations_split[
    df_albums_collaborations_split['performer'].str.contains(
        r'&|[Aa]nd|[Ff]eat|[Ff]t\.|[Ww]ith|X|,', 
        na=False, 
        regex=True
    )
]

print(f"\nPotential unsplit collaborations in ALBUMS: {len(albums_collabs):,}")
print(f"Unique performers: {albums_collabs['performer'].nunique():,}")

if len(albums_collabs) > 0:
    print("\nSample potentially unsplit albums performers:")
    print(albums_collabs['performer'].value_counts().head(50))

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"Songs with potential unsplit collabs: {len(songs_collabs):,}")
print(f"Albums with potential unsplit collabs: {len(albums_collabs):,}")


SONGS DATAFRAME - Checking for unsplit collaborations

Potential unsplit collaborations in SONGS: 3,969
Unique performers: 1,660

Sample potentially unsplit songs performers:
performer
Ariana Grande                        95
Tyler, The Creator                   45
Andy Williams                        40
Barbra Streisand                     40
Gladys Knight And The Pips           38
James Brown And The Famous Flames    37
Bobby Bland                          34
XXXTENTACION                         34
Kool & The Gang                      32
Earth, Wind & Fire                   31
PARTYNEXTDOOR                        28
Brooks & Dunn                        27
Jan & Dean                           24
Luther Vandross                      24
Miranda Lambert                      24
Gene Chandler                        23
Brian Hyland                         22
Zac Brown Band                       22
Huey Lewis & The News                21
Jr. Walker & The All Stars           21
Herb Alpert & T

In [15]:
# Exact match on normalized names
df_exact_match = df_songs_collaborations_split.merge(
    df_albums_collaborations_split,
    on='performer_normalized',
    how='inner',
    suffixes=('_song', '_album')
)

print("="*60)
print("EXACT MATCH RESULTS")
print("="*60)
print(f"Total matched rows: {len(df_exact_match):,}")
print(f"Unique performers matched: {df_exact_match['performer_normalized'].nunique():,}")
print("="*60)

# Show sample matches
print("\nSample exact matches:")
sample = df_exact_match[['performer_song', 'title_song', 'title_album']].head(15)
print(sample.to_string(index=False))

# Show column names
print(f"\nColumns in merged dataframe: {list(df_exact_match.columns)}")


EXACT MATCH RESULTS
Total matched rows: 322,385
Unique performers matched: 4,592

Sample exact matches:
performer_song             title_song                                     title_album
 Conway Twitty It's Only Make Believe                           I Love You More Today
 Conway Twitty It's Only Make Believe                                   Hello Darlin'
 Conway Twitty It's Only Make Believe                     How Much More Can She Stand
 Conway Twitty It's Only Make Believe                               Fifteen Years Ago
 Conway Twitty It's Only Make Believe     I Wonder What She'll Think About Me Leaving
 Conway Twitty It's Only Make Believe                      I Can't See Me Without You
 Conway Twitty It's Only Make Believe You've Never Been This Far Before / Baby's Gone
 Conway Twitty It's Only Make Believe                                Southern Comfort
 Conway Twitty It's Only Make Believe                                   Final Touches
 Conway Twitty It's Only Make Believ

In [16]:
# Get performers that were matched
matched_performers = df_exact_match['performer_normalized'].unique()

# Find unmatched songs
df_songs_unmatched = df_songs_collaborations_split[
    ~df_songs_collaborations_split['performer_normalized'].isin(matched_performers)
]

# Find unmatched albums
df_albums_unmatched = df_albums_collaborations_split[
    ~df_albums_collaborations_split['performer_normalized'].isin(matched_performers)
]

print("="*60)
print("UNMATCHED RECORDS")
print("="*60)
print(f"Unmatched songs rows: {len(df_songs_unmatched):,}")
print(f"Unmatched songs performers: {df_songs_unmatched['performer_normalized'].nunique():,}")
print()
print(f"Unmatched albums rows: {len(df_albums_unmatched):,}")
print(f"Unmatched albums performers: {df_albums_unmatched['performer_normalized'].nunique():,}")
print("="*60)

# Show some examples of unmatched performers
print("\nSample unmatched SONGS performers:")
print(df_songs_unmatched['performer'].value_counts().head(10))

print("\nSample unmatched ALBUMS performers:")
print(df_albums_unmatched['performer'].value_counts().head(10))


UNMATCHED RECORDS
Unmatched songs rows: 8,067
Unmatched songs performers: 4,837

Unmatched albums rows: 14,440
Unmatched albums performers: 5,440

Sample unmatched SONGS performers:
performer
Glee Cast                            206
Connie Francis                        53
The Jordanaires                       53
Bobby Darin                           40
James Brown And The Famous Flames     37
Bobby Rydell                          28
Johnny Tillotson                      25
Jan & Dean                            24
The Shirelles                         24
Mary Wells                            23
Name: count, dtype: int64

Sample unmatched ALBUMS performers:
performer
Soundtrack                                            1601
Original Broadway Cast Recording                        55
Kidz Bop Kids                                           54
Original Cast                                           40
Phish                                                   38
Bill & Gloria Gaither And Thei

In [17]:
# Check how many unique performers had top 20 hits
songs_top20_performers = df_songs_unmatched[
    df_songs_unmatched['peak_pos'] <= 20
]['performer_normalized'].nunique()

albums_top20_performers = df_albums_unmatched[
    df_albums_unmatched['peak_pos'] <= 20
]['performer_normalized'].nunique()

print(f"Unmatched songs performers with top 20 hits: {songs_top20_performers:,}")
print(f"Unmatched albums performers with top 20 hits: {albums_top20_performers:,}")
print(f"Total comparisons needed: {songs_top20_performers * albums_top20_performers:,}")


Unmatched songs performers with top 20 hits: 1,103
Unmatched albums performers with top 20 hits: 598
Total comparisons needed: 659,594


In [18]:
from rapidfuzz import process, fuzz

# Filter to only top 20 artists
df_songs_top20 = df_songs_unmatched[df_songs_unmatched['peak_pos'] <= 20]
df_albums_top20 = df_albums_unmatched[df_albums_unmatched['peak_pos'] <= 20]

# Get unique performers
songs_performers_top20 = df_songs_top20['performer_normalized'].unique()
albums_performers_top20 = df_albums_top20['performer_normalized'].unique()

print(f"Fuzzy matching {len(songs_performers_top20):,} songs performers against {len(albums_performers_top20):,} albums performers...")
print(f"Threshold: 90% similarity\n")

fuzzy_matches = []

for i, song_performer in enumerate(songs_performers_top20):
    if i % 100 == 0:
        print(f"Progress: {i}/{len(songs_performers_top20)}")
    
    # Find best match in albums
    match = process.extractOne(
        song_performer, 
        albums_performers_top20, 
        scorer=fuzz.ratio
    )
    
    if match and match[1] >= 90:  # match[1] is the similarity score
        fuzzy_matches.append({
            'song_performer': song_performer,
            'album_performer': match[0],
            'similarity': match[1]
        })

fuzzy_matches_df = pd.DataFrame(fuzzy_matches)

print(f"\n{'='*60}")
print(f"✅ Fuzzy matches found: {len(fuzzy_matches_df):,}")
print(f"{'='*60}")

# Show top matches
if len(fuzzy_matches_df) > 0:
    print("\nTop fuzzy matches:")
    print(fuzzy_matches_df.sort_values('similarity', ascending=False).head(20))
    
    # Show borderline matches (90-95% similarity) for review
    borderline = fuzzy_matches_df[
        (fuzzy_matches_df['similarity'] >= 90) & 
        (fuzzy_matches_df['similarity'] < 95)
    ]
    print(f"\n⚠️ Borderline matches to review: {len(borderline)}")
    if len(borderline) > 0:
        print(borderline.sort_values('similarity', ascending=False))


Fuzzy matching 1,103 songs performers against 598 albums performers...
Threshold: 90% similarity

Progress: 0/1103
Progress: 100/1103
Progress: 200/1103
Progress: 300/1103
Progress: 400/1103
Progress: 500/1103
Progress: 600/1103
Progress: 700/1103
Progress: 800/1103
Progress: 900/1103
Progress: 1000/1103
Progress: 1100/1103

✅ Fuzzy matches found: 5

Top fuzzy matches:
                song_performer              album_performer  similarity
3               vonda shepard)                vonda shepard   96.296296
1                  lipps, inc.                   lipps inc.   95.238095
4  missy "misdemeanor" elliott  missy 'misdemeanor' elliott   92.592593
0      evelyn "champagne" king      evelyn 'champagne' king   91.304348
2          bob & doug mckenzie        bob and doug mckenzie   90.000000

⚠️ Borderline matches to review: 3
                song_performer              album_performer  similarity
4  missy "misdemeanor" elliott  missy 'misdemeanor' elliott   92.592593
0      evelyn "c

In [19]:
# Check fuzzy matches against MusicBrainz and update dataframes
print("Checking fuzzy matches against MusicBrainz and updating dataframes...\n")

canonical_matches = []
updates_made = 0

for idx, row in fuzzy_matches_df.iterrows():
    song_name = row['song_performer']
    album_name = row['album_performer']
    
    print(f"Checking pair {idx+1}/5:")
    print(f"  Song version:  '{song_name}'")
    print(f"  Album version: '{album_name}'")
    
    # Check song version
    song_verified, song_confidence = check_musicbrainz_variants(song_name)
    
    # Check album version
    album_verified, album_confidence = check_musicbrainz_variants(album_name)
    
    print(f"  Song verified:  {song_verified} (confidence: {song_confidence})")
    print(f"  Album verified: {album_verified} (confidence: {album_confidence})")
    
    # Determine canonical name
    if song_verified and not album_verified:
        canonical = song_name
        # Update albums dataframe to use song version
        df_albums_collaborations_split.loc[
            df_albums_collaborations_split['performer_normalized'] == album_name,
            'performer_normalized'
        ] = canonical
        print(f"  → Updated ALBUMS: '{album_name}' → '{canonical}'")
        updates_made += 1
        
    elif album_verified and not song_verified:
        canonical = album_name
        # Update songs dataframe to use album version
        df_songs_collaborations_split.loc[
            df_songs_collaborations_split['performer_normalized'] == song_name,
            'performer_normalized'
        ] = canonical
        print(f"  → Updated SONGS: '{song_name}' → '{canonical}'")
        updates_made += 1
        
    elif song_verified and album_verified:
        canonical = song_name
        # Both valid, standardize to song version
        df_albums_collaborations_split.loc[
            df_albums_collaborations_split['performer_normalized'] == album_name,
            'performer_normalized'
        ] = canonical
        print(f"  → Both valid, standardized ALBUMS to: '{canonical}'")
        updates_made += 1
        
    else:
        canonical = album_name
        # Neither verified, use album as default
        df_songs_collaborations_split.loc[
            df_songs_collaborations_split['performer_normalized'] == song_name,
            'performer_normalized'
        ] = canonical
        print(f"  → Neither verified, standardized SONGS to: '{canonical}'")
        updates_made += 1
    
    print(f"  ✓ Canonical name: '{canonical}'\n")
    
    canonical_matches.append({
        'song_performer': song_name,
        'album_performer': album_name,
        'canonical_name': canonical,
        'song_verified': song_verified,
        'album_verified': album_verified
    })
    
    time.sleep(1.5)  # Rate limiting

canonical_df = pd.DataFrame(canonical_matches)

print("="*60)
print("UPDATES COMPLETE")
print("="*60)
print(f"Total updates made: {updates_made}")
print("\nCanonical names:")
print(canonical_df[['song_performer', 'album_performer', 'canonical_name']].to_string(index=False))
print("\n✅ Both dataframes updated with canonical names from MusicBrainz")


Checking fuzzy matches against MusicBrainz and updating dataframes...

Checking pair 1/5:
  Song version:  'evelyn "champagne" king'
  Album version: 'evelyn 'champagne' king'
  Song verified:  False (confidence: 0)
  Album verified: True (confidence: 100)
  → Updated SONGS: 'evelyn "champagne" king' → 'evelyn 'champagne' king'
  ✓ Canonical name: 'evelyn 'champagne' king'

Checking pair 2/5:
  Song version:  'lipps, inc.'
  Album version: 'lipps inc.'
  Song verified:  True (confidence: 100)
  Album verified: True (confidence: 100)
  → Both valid, standardized ALBUMS to: 'lipps, inc.'
  ✓ Canonical name: 'lipps, inc.'

Checking pair 3/5:
  Song version:  'bob & doug mckenzie'
  Album version: 'bob and doug mckenzie'
  Song verified:  True (confidence: 100)
  Album verified: True (confidence: 100)
  → Both valid, standardized ALBUMS to: 'bob & doug mckenzie'
  ✓ Canonical name: 'bob & doug mckenzie'

Checking pair 4/5:
  Song version:  'vonda shepard)'
  Album version: 'vonda shepard'


In [20]:
# Re-run exact match with updated names
df_exact_match_v2 = df_songs_collaborations_split.merge(
    df_albums_collaborations_split,
    on='performer_normalized',
    how='inner',
    suffixes=('_song', '_album')
)

print(f"New exact matches: {len(df_exact_match_v2):,} (was {len(df_exact_match):,})")
print(f"Additional matches gained: {len(df_exact_match_v2) - len(df_exact_match):,}")


New exact matches: 322,464 (was 322,385)
Additional matches gained: 79


In [21]:
# Re-run exact match with updated canonical names
df_exact_match_v2 = df_songs_collaborations_split.merge(
    df_albums_collaborations_split,
    on='performer_normalized',
    how='inner',
    suffixes=('_song', '_album')
)

print("="*60)
print("UPDATED EXACT MATCH RESULTS")
print("="*60)
print(f"Previous exact matches: {len(df_exact_match):,}")
print(f"New exact matches:      {len(df_exact_match_v2):,}")
print(f"Additional matches:     {len(df_exact_match_v2) - len(df_exact_match):,}")
print()
print(f"Unique performers matched: {df_exact_match_v2['performer_normalized'].nunique():,}")
print("="*60)

# Show the newly matched records
if len(df_exact_match_v2) > len(df_exact_match):
    print("\nNewly matched performers (from fuzzy matching):")
    # Find which performers are new
    old_performers = set(df_exact_match['performer_normalized'].unique())
    new_performers = set(df_exact_match_v2['performer_normalized'].unique())
    added_performers = new_performers - old_performers
    
    for performer in added_performers:
        count = len(df_exact_match_v2[df_exact_match_v2['performer_normalized'] == performer])
        print(f"  - {performer}: {count} matched rows")


UPDATED EXACT MATCH RESULTS
Previous exact matches: 322,385
New exact matches:      322,464
Additional matches:     79

Unique performers matched: 4,597

Newly matched performers (from fuzzy matching):
  - evelyn 'champagne' king: 35 matched rows
  - missy 'misdemeanor' elliott: 36 matched rows
  - vonda shepard: 3 matched rows
  - bob & doug mckenzie: 1 matched rows
  - lipps, inc.: 4 matched rows


In [22]:
df_exact_match_v2.head()

,title_song,performer_song,peak_pos_song,wks_on_chart_song,first_charting_year_song,original_performer_song,performer_normalized,title_album,performer_album,peak_pos_album,wks_on_chart_album,first_charting_year_album,original_performer_album
0,It's Only Make Believe,Conway Twitty,1,21,1958,NaN,conway twitty,I Love You More Today,Conway Twitty,161,3,1969,NaN
1,It's Only Make Believe,Conway Twitty,1,21,1958,NaN,conway twitty,Hello Darlin',Conway Twitty,65,26,1970,NaN
2,It's Only Make Believe,Conway Twitty,1,21,1958,NaN,conway twitty,How Much More Can She Stand,Conway Twitty,91,9,1971,NaN
3,It's Only Make Believe,Conway Twitty,1,21,1958,NaN,conway twitty,Fifteen Years Ago,Conway Twitty,140,7,1971,NaN
4,It's Only Make Believe,Conway Twitty,1,21,1958,NaN,conway twitty,I Wonder What She'll Think About Me Leaving,Conway Twitty,142,8,1971,NaN


In [23]:
# Get the list of performers who appear in BOTH datasets
matched_performers = set(df_songs_collaborations_split['performer_normalized'].unique()) & \
                    set(df_albums_collaborations_split['performer_normalized'].unique())

print(f"Performers in BOTH songs and albums: {len(matched_performers):,}")

# Filter songs to only matched performers
df_songs_matched = df_songs_collaborations_split[
    df_songs_collaborations_split['performer_normalized'].isin(matched_performers)
].copy()

# Filter albums to only matched performers
df_albums_matched = df_albums_collaborations_split[
    df_albums_collaborations_split['performer_normalized'].isin(matched_performers)
].copy()

# Add a 'type' column to distinguish them
df_songs_matched['record_type'] = 'song'
df_albums_matched['record_type'] = 'album'

# Rename 'title' column to be consistent (assuming both have 'title')
# This ensures column names match when stacking

# Stack them together
df_combined = pd.concat([df_songs_matched, df_albums_matched], ignore_index=True)

print("="*60)
print("COMBINED STACKED DATAFRAME")
print("="*60)
print(f"Total rows: {len(df_combined):,}")
print(f"  Songs: {len(df_songs_matched):,}")
print(f"  Albums: {len(df_albums_matched):,}")
print(f"Unique performers: {df_combined['performer_normalized'].nunique():,}")
print("="*60)

# Show example for one artist
sample_artist = df_combined['performer_normalized'].value_counts().head(1).index[0]
sample_data = df_combined[df_combined['performer_normalized'] == sample_artist]

print(f"\nExample: {sample_artist}")
print(f"  Songs: {len(sample_data[sample_data['record_type'] == 'song'])}")
print(f"  Albums: {len(sample_data[sample_data['record_type'] == 'album'])}")
print(f"  Total: {len(sample_data)}")

# Show sample rows
print(f"\nSample rows:")
print(df_combined[['performer', 'title', 'record_type', 'peak_pos', 'wks_on_chart']].head(10))


Performers in BOTH songs and albums: 4,597
COMBINED STACKED DATAFRAME
Total rows: 53,690
  Songs: 29,291
  Albums: 24,399
Unique performers: 4,597

Example: various artists
  Songs: 4
  Albums: 1608
  Total: 1612

Sample rows:
             performer                   title record_type  peak_pos  \
0        Conway Twitty  It's Only Make Believe        song         1   
1         Ricky Nelson        Poor Little Fool        song         1   
2        The Chipmunks       The Chipmunk Song        song         1   
3        David Seville       The Chipmunk Song        song         1   
4    The Kingston Trio              Tom Dooley        song         1   
5  The Everly Brothers                Bird Dog        song         2   
6       Ritchie Valens                   Donna        song         2   
7          Bobby Helms        Jingle Bell Rock        song         2   
8  The Everly Brothers                Problems        song         2   
9        Elvis Presley       Hard Headed Woman       

In [24]:
df_combined.head()

,title,performer,peak_pos,wks_on_chart,first_charting_year,original_performer,performer_normalized,record_type
0,It's Only Make Believe,Conway Twitty,1,21,1958,NaN,conway twitty,song
1,Poor Little Fool,Ricky Nelson,1,11,1958,NaN,ricky nelson,song
2,The Chipmunk Song,The Chipmunks,1,13,1958,The Chipmunks With David Seville,the chipmunks,song
3,The Chipmunk Song,David Seville,1,13,1958,The Chipmunks With David Seville,david seville,song
4,Tom Dooley,The Kingston Trio,1,21,1958,NaN,the kingston trio,song


In [25]:
# Save combined dataframe to CSV
output_path = '/Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_songs_and_albums_combined.csv'

df_combined.to_csv(output_path, index=False)

print("✅ Saved combined dataframe to CSV")
print(f"📁 File: {output_path}")
print(f"📊 Total rows: {len(df_combined):,}")
print(f"   Songs: {len(df_combined[df_combined['record_type'] == 'song']):,}")
print(f"   Albums: {len(df_combined[df_combined['record_type'] == 'album']):,}")
print(f"🎤 Unique performers: {df_combined['performer_normalized'].nunique():,}")
print(f"📋 Columns: {len(df_combined.columns)}")


✅ Saved combined dataframe to CSV
📁 File: /Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_songs_and_albums_combined.csv
📊 Total rows: 53,690
   Songs: 29,291
   Albums: 24,399
🎤 Unique performers: 4,597
📋 Columns: 8


In [26]:
df_songs_collaborations_split.head()

,title,performer,peak_pos,wks_on_chart,first_charting_year,original_performer,performer_normalized
0,It's All In The Game,Tommy Edwards,1,22,1958,NaN,tommy edwards
1,It's Only Make Believe,Conway Twitty,1,21,1958,NaN,conway twitty
2,Little Star,The Elegants,1,17,1958,NaN,the elegants
3,Nel Blu Dipinto Di Blu (Volaré),Domenico Modugno,1,16,1958,NaN,domenico modugno
4,Poor Little Fool,Ricky Nelson,1,11,1958,NaN,ricky nelson


In [27]:
# Save cleaned songs dataframe
songs_output_path = '/Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_hot100_songs_cleaned.csv'
df_songs_collaborations_split.to_csv(songs_output_path, index=False)

print("✅ Saved cleaned SONGS dataframe")
print(f"📁 File: {songs_output_path}")
print(f"📊 Rows: {len(df_songs_collaborations_split):,}")
print(f"🎤 Unique performers: {df_songs_collaborations_split['performer_normalized'].nunique():,}")
print(f"📋 Columns: {list(df_songs_collaborations_split.columns)}")

print("\n" + "="*60 + "\n")

# Save cleaned albums dataframe
albums_output_path = '/Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_200_albums_cleaned.csv'
df_albums_collaborations_split.to_csv(albums_output_path, index=False)

print("✅ Saved cleaned ALBUMS dataframe")
print(f"📁 File: {albums_output_path}")
print(f"📊 Rows: {len(df_albums_collaborations_split):,}")
print(f"🎤 Unique performers: {df_albums_collaborations_split['performer_normalized'].nunique():,}")
print(f"📋 Columns: {list(df_albums_collaborations_split.columns)}")


✅ Saved cleaned SONGS dataframe
📁 File: /Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_hot100_songs_cleaned.csv
📊 Rows: 37,337
🎤 Unique performers: 9,429
📋 Columns: ['title', 'performer', 'peak_pos', 'wks_on_chart', 'first_charting_year', 'original_performer', 'performer_normalized']


✅ Saved cleaned ALBUMS dataframe
📁 File: /Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_200_albums_cleaned.csv
📊 Rows: 38,823
🎤 Unique performers: 10,032
📋 Columns: ['title', 'performer', 'peak_pos', 'wks_on_chart', 'first_charting_year', 'original_performer', 'performer_normalized']


In [28]:
# Create combined dataframe with EVERYONE (songs-only, albums-only, and both)
df_songs_all = df_songs_collaborations_split.copy()
df_albums_all = df_albums_collaborations_split.copy()

# Add record type column to distinguish songs from albums
df_songs_all['record_type'] = 'song'
df_albums_all['record_type'] = 'album'

# Combine ALL songs and ALL albums (stacked vertically)
df_combined_all = pd.concat([df_songs_all, df_albums_all], ignore_index=True)

print("="*60)
print("COMBINED DATAFRAME (EVERYONE)")
print("="*60)
print(f"Total rows: {len(df_combined_all):,}")
print(f"  Songs: {len(df_songs_all):,}")
print(f"  Albums: {len(df_albums_all):,}")
print(f"Total unique performers: {df_combined_all['performer_normalized'].nunique():,}")
print("="*60)

# Breakdown by performer type
performers_both = set(df_songs_all['performer_normalized'].unique()) & \
                 set(df_albums_all['performer_normalized'].unique())
performers_songs_only = set(df_songs_all['performer_normalized'].unique()) - \
                        set(df_albums_all['performer_normalized'].unique())
performers_albums_only = set(df_albums_all['performer_normalized'].unique()) - \
                         set(df_songs_all['performer_normalized'].unique())

print(f"\nPerformer breakdown:")
print(f"  In BOTH songs & albums: {len(performers_both):,}")
print(f"  Songs ONLY: {len(performers_songs_only):,}")
print(f"  Albums ONLY: {len(performers_albums_only):,}")

# Show sample
print(f"\nSample rows:")
print(df_combined_all[['performer', 'title', 'record_type', 'peak_pos', 'wks_on_chart']].head(10))


COMBINED DATAFRAME (EVERYONE)
Total rows: 76,160
  Songs: 37,337
  Albums: 38,823
Total unique performers: 14,864

Performer breakdown:
  In BOTH songs & albums: 4,597
  Songs ONLY: 4,832
  Albums ONLY: 5,435

Sample rows:
          performer                            title record_type  peak_pos  \
0     Tommy Edwards             It's All In The Game        song         1   
1     Conway Twitty           It's Only Make Believe        song         1   
2      The Elegants                      Little Star        song         1   
3  Domenico Modugno  Nel Blu Dipinto Di Blu (Volaré)        song         1   
4      Ricky Nelson                 Poor Little Fool        song         1   
5      The Platters          Smoke Gets In Your Eyes        song         1   
6       Lloyd Price                      Stagger Lee        song         1   
7     The Chipmunks                The Chipmunk Song        song         1   
8     David Seville                The Chipmunk Song        song         1 

In [29]:
df_combined_all.head()

,title,performer,peak_pos,wks_on_chart,first_charting_year,original_performer,performer_normalized,record_type
0,It's All In The Game,Tommy Edwards,1,22,1958,NaN,tommy edwards,song
1,It's Only Make Believe,Conway Twitty,1,21,1958,NaN,conway twitty,song
2,Little Star,The Elegants,1,17,1958,NaN,the elegants,song
3,Nel Blu Dipinto Di Blu (Volaré),Domenico Modugno,1,16,1958,NaN,domenico modugno,song
4,Poor Little Fool,Ricky Nelson,1,11,1958,NaN,ricky nelson,song


In [30]:
# Save combined dataframe to CSV
combined_output_path = '/Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_songs_and_albums_combined_all.csv'

df_combined_all.to_csv(combined_output_path, index=False)

print("✅ Saved combined dataframe to CSV")
print(f"📁 File: {combined_output_path}")
print(f"📊 Total rows: {len(df_combined_all):,}")
print(f"   Songs: {len(df_combined_all[df_combined_all['record_type'] == 'song']):,}")
print(f"   Albums: {len(df_combined_all[df_combined_all['record_type'] == 'album']):,}")
print(f"🎤 Unique performers: {df_combined_all['performer_normalized'].nunique():,}")
print(f"📋 Columns: {len(df_combined_all.columns)}")


✅ Saved combined dataframe to CSV
📁 File: /Users/jamesemcnally/Dropbox/DSBC/Spring 2026/billboard_songs_and_albums_combined_all.csv
📊 Total rows: 76,160
   Songs: 37,337
   Albums: 38,823
🎤 Unique performers: 14,864
📋 Columns: 8
